In [1]:
import os
import shutil
import inspect
import time
import numpy as np
import torch
import torch.nn as nn
from tensorflow.python.keras import backend
#from tensorflow.python.data.benchmarks import mnist
#from tensorflow.python.keras.utils import to_categorical

from snntoolbox.bin.run import main
from snntoolbox.utils.utils import import_configparser
from tests.parsing.models.pytorch import Model

2024-11-12 08:33:41.792423: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-12 08:33:41.811640: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Pytorch to Keras parser needs image_data_format == channel_first.
backend.set_image_data_format('channels_first')

# Define path where model and output files will be stored.
# The user is responsible for cleaning up this temporary directory.
path_wd = os.path.abspath(os.path.join(os.path.dirname(os.path.realpath(
    os.path.abspath(''))), '..', 'temp', str(time.time())))
os.makedirs(path_wd)

# Create a config file with experimental setup for SNN Toolbox.
configparser = import_configparser()
config = configparser.ConfigParser()

config['paths'] = {
    'path_wd': path_wd,             # Path to model.
    'dataset_path': path_wd,        # Path to dataset.
	'filename_ann': './saved_models/resnet20_cifar10.pth' # Name of input model.
}

config['tools'] = {
    'evaluate_ann': True, # Test ANN on dataset before conversion.
    'normalize': False # Normalize weights for full dynamic range.
}

config['simulation'] = {
    'simulator': 'INI',             # Chooses execution backend of SNN toolbox.
    'duration': 50,                 # Number of time steps to run each sample.
    'num_to_test': 100,             # How many test samples to run.
    'batch_size': 128,               # Batch size for simulation.
    'keras_backend': 'tensorflow'   # Which keras backend to use.
}

config['input'] = {
    'model_lib': 'pytorch'          # Input model is defined in pytorch.
}

config['output'] = {
    'plot_vars': {                  # Various plots (slows down simulation).
        'spiketrains',              # Leave section empty to turn off plots.
        'spikerates',
        'activations',
        'correlation',
        'v_mem',
        'error_t'}
}

# Store config file.
config_filepath = os.path.join(path_wd, 'config')
with open(config_filepath, 'w') as configfile:
    config.write(configfile)

# Need to copy model definition over to ``path_wd`` (needs to be in same dir as
# the weights saved above).
source_path = inspect.getfile(Model)
shutil.copyfile(source_path, os.path.join(path_wd, 'modules.py'))

# RUN SNN TOOLBOX #
###################

main(config_filepath)

RuntimeWarning: Data set file 'x_test.npz' or 'y_test.npz' was not found in specified data set path /home/liamb/temp/1731371623.0417182.